# LAB 1 &ndash; Visual cryptography

In order to make sense of some of the things we mentioned in class, it is useful to see them in action on image files; for we, as humans, are quite good at visually detecting patterns in an array of pixels. Use SHIFT+ENTER (or the "Play" button above) to evaluate a cell.

First thing to do: double-click on this text cell to write your <b style="color: red">BEQUART Valentin, BERTIN Théo, BRACONNIER Martin, QUAMENDA Quentin</b>.

## 1) First attempt

Here is a picture of Bletchley Park, where modern computing was born as Alan Turing and thousands of cryptananalysists worked on the breaking of the German codes during WWII:

In [ ]:
load("Image.sage")

m = LoadImage("bletchley.tiff")
m                                # displays it

Internally, it's just a $288 \times 460$ matrix of pixels, each of which is colored according to a RGB triple of integers between 0 and 255:

In [ ]:
m.data

Recall that the goal of encryption would be to reversibly make it look like a random image of the same size just like this one:

In [ ]:
RandomImage(288,460)

As a single pixel is encoded on $3 \times 8 = 24$ bits, a $4 \times 4$ block of pixels is encoded on $16 \times 24 = 384$ bits, which give us more than enough room to store a secure symmetric encryption key.

In [ ]:
k = RandomImage(4,4); k

In [ ]:
pad = Image(numpy.tile(k.data, (288/4,460/4,1)))
pad

Et voilà! Here's the picture, encrypted by a 384-bit key:

In [ ]:
m + pad    # pixel values are xor-ed together

<b>To do:</b> Discuss the result. Would you say that this cipher provides perfect secrecy? 
On ne pourra pas retrouver l'endroit de la photo, mais on peut deviner des arbres et un batiment sur la photo, donc secrecy non parfaite.


## 2) One-time pad

Go through the same process, this time with a genuine randomly generated one-time pad as large as the image to encrypt.

In [ ]:
k = RandomImage(288,460)
k

What's the key-length this time?

In [ ]:
nb = 288*460*24;
nb

Verify that the cipher decrypts correctly, <i>i.e.</i> play both the roles of Alice and Bob to see that everything works as it should. Also make sure to take a look at what Eve actually sees on the insecure channel.

In [ ]:
a = m + k;
a 

In [ ]:
a + k

## 3) Two-time pad

Wow, this works well. In the excitation of the moment, I started encrypting all my images... but forgot to apply one of the most important usage rules of the one-time pad when I encrypted the following two images. Can you find out what they were?

In [ ]:
c1 = LoadImage("c1.tiff"); c1

In [ ]:
c2 = LoadImage("c2.tiff"); c2

In [ ]:
c1 + c2

On est donc capable de deviner les images de départ, Bob et Alice, en appliquant le principe : (C1 = m1 + k et c2 = m2 + k sachant que les + sont des XOR, donc en faisant c1 + c2, les k se suppriment, on obtient une superposition des deux images de départ).

## 4) Weak random number generator

After realizing my mistake, I decided to generate pads from a 128-bit key by using it as a seed for a linear congruence generator modulo the following 128-bit prime:

In [ ]:
p = 340282366920938463463374607431768211507

is_prime(p)

Since such a PRNG is characterized by two 128-bit integers $a$ and $b$ which were chosen at random, there should be more than enough entropy to protect my 128-bit key, right? Here are the first few outputs from the LCG:

217692597915196650809181220736554072509,
101697276836279744146238049998237762682,
265181937610212296333751058245677871006,
84171444745593992579687306707926595136,
12455596861516498286468598807461112654,
...

prove me wrong by finding out which seed (<i>i.e.</i> "private key") was used.

Nous faisons le choix de considérer les coefficients a et b comme ceux d'une fonction affine, nous appliquons donc la méthode de recherche des coefficients de ce type de ce type de fonction.

In [ ]:
x1 = 217692597915196650809181220736554072509;
x2 = 101697276836279744146238049998237762682;
fx1 = 101697276836279744146238049998237762682;
fx2 = 265181937610212296333751058245677871006;
a = (fx2 - fx1) / (x2 - x1);
a


In [ ]:
b = x2 - (a*x1);
b

Effectuons un test pour vérifier qu'en rentrant la deuxième valeur donnée dans l'énoncé, nous retrouvons bien la troisième

In [ ]:
r = a*x2 + b
r == fx2

Il ne reste plus qu'à retrouver la racine, en précisant que la sortie de celle-ci dans la fonction se trouve être x1, soit la première valeur donnée dans l'énoncé.

In [ ]:
x0 = (x1 - b)/a
x0%p

Nous pensons avoir trouvé notre racine, une dernière vérification s'impose:

In [ ]:
(a*x1+b) % p

## 5) Genuine stream cipher

Ok, now it's time to start doing things properly. We will use the still-standard-albeit-somewhat-deprecated RC4 pseudo-random number generator to generate a key-stream from a 128-bit key. We first aquire 128 bits of "real" random data from the entropy pool of the system. 

In [ ]:
k = os.urandom(16)      # 16 bytes of entropy coming from /dev/urandom
k.encode('hex')

In [ ]:
load("RC4.sage")
prng = RC4(k)

<b>prng</b> is now an instance of the RC4 PRNG initialized with seed $k$; you can get its successive outputs by typing <b>prng.next()</b>.

Use this to generate of pseudo-random pad from $k$, and then encrypt the Bletchley Park picture with it. Make sure that Bob will be able to decrypt it knowing only $k$. 

In [ ]:
prng.next()

In [ ]:
#original image

m = LoadImage("bletchley.tiff")
m

In [ ]:
#encrypting

c = m
k = os.urandom(16)
prng = RC4(k)

for i in [0..287] :
    for j in [0..459] :
        for l in [0..2] :
            c.data[i][j][l] += prng.next()
c

In [ ]:
#decrypting

prng = RC4(k)
d = c

for i in [0..287] :
    for j in [0..459] :
        for l in [0..2] :
            d.data[i][j][l] -= prng.next()
d